# #Sentiment Analysis 

In [1]:
import re
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.metrics import classification_report

In [2]:
#Load the IMDB dataset. You can load it using pandas as dataframe
data = pd.read_csv("IMDB Dataset.csv")
print(data.head())

                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


# Preprocessing


In [3]:
#stopwords = code for downloading stop words through nltk
import nltk

nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize

nltk.download('stopwords')
from nltk.corpus import stopwords 
stopword = stopwords.words('english')

from nltk.stem import PorterStemmer
porter_stemmer = PorterStemmer()

from nltk.stem import WordNetLemmatizer
nltk.download('wordnet') 
lemmatizer = WordNetLemmatizer()

from bs4 import BeautifulSoup 

for i, row in data.iterrows():

    sentence = row['review']
    
    # remove html tags
    sentence = BeautifulSoup(sentence, 'lxml').text

    # remove urls
    sentence = re.sub('https\S+', '', sentence)
    
    # removing punctuations : how? -> how 
    sentence = re.sub(r'[^\w\s]', ' ', sentence)

    tokens = word_tokenize(sentence, language='english')
    # convert to lower case and remove non alphanumeric words
    tokens = [token.lower() for token in tokens if token.isalnum() and token not in stopword] 

    # perform stemming and lemmatization
    tokens = [porter_stemmer.stem(lemmatizer.lemmatize(token)) for token in tokens]

    data.at[i, 'review'] = " ".join(tokens)


[nltk_data] Downloading package punkt to /home/shubham/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/shubham/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/shubham/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [17]:
# Statistics of data like avg length of sentence ,
word_count = 0
sent_count = 0
for sentence in data['review']:
    words = sentence.split()
    word_count = word_count + len(words)
    sent_count = sent_count + 1
print(f'The average length of sentence is : {word_count/sent_count}')


# Proposition of data w.r.t class labels
for clas in data['sentiment'].value_counts().keys():
    freq = data['sentiment'].value_counts()[clas]
    print(f'Class {clas} composes {freq} samples out of {len(data)} in dataset.')


The average length of sentence is : 131.11266
Class negative composes 25000 samples out of 50000 in dataset.
Class positive composes 25000 samples out of 50000 in dataset.


# Naive Bayes classifier

In [18]:
# get reviews column from df
reviews = data['review']

# get labels column from df
labels = data['sentiment']

In [6]:
# Use label encoder to encode labels. Convert to 0/1
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(labels)

# print(encoded_labels)

In [16]:
# Split the data into train and test (80% - 20%). 
# Use stratify in train_test_split so that both train and test have similar ratio of positive and negative samples.
from sklearn.model_selection import StratifiedShuffleSplit
for train_index, test_index in StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0).split(reviews, encoded_labels):
      train_sentences, test_sentences, train_labels, test_labels = reviews[train_index], reviews[test_index], encoded_labels[train_index], encoded_labels[test_index]

In [8]:
# Use Count vectorizer to get frequency of the words
from sklearn.feature_extraction.text import CountVectorizer


# max_features parameter : If not None, build a vocabulary that only consider the top max_features ordered by term frequency across the corpus.
vec_train_pos = CountVectorizer(max_features = 3000)
X_train_pos = vec_train_pos.fit_transform(train_sentences[train_labels == 1].tolist())

vec_train_neg = CountVectorizer(max_features = 3000)
X_train_neg = vec_train_neg.fit_transform(train_sentences[train_labels == 0].tolist())

In [9]:
# Use laplace smoothing for words in test set not present in vocab of train set
train_word_list_pos = vec_train_pos.get_feature_names()
train_count_list_pos = X_train_pos.toarray().sum(axis=0).tolist()

train_word_list_neg = vec_train_neg.get_feature_names()
train_count_list_neg = X_train_neg.toarray().sum(axis=0).tolist()

pos_word_prob = dict(zip(train_word_list_pos, train_count_list_pos))
neg_word_prob = dict(zip(train_word_list_neg, train_count_list_neg))

In [ ]:
import math
# Build the model. Don't use the model from sklearn
def modelNB(sentence):
    pos_prob = 0
    neg_prob = 0
    sum_pos_word = sum(list(pos_word_prob.values()))
    sum_neg_word = sum(list(neg_word_prob.values()))

  # print(type(sentence))
    for word in sentence.split():
    # print(word)
        if word in pos_word_prob.keys():
            pos_prob += math.log((pos_word_prob[word]+1)/(sum_pos_word+len(pos_word_prob)))
        else : 
            pos_prob += math.log(1/(sum_pos_word+len(pos_word_prob)))
    
        if word in neg_word_prob.keys():
            neg_prob += math.log((neg_word_prob[word]+1)/(sum_neg_word+len(neg_word_prob)))
        else : 
            neg_prob += math.log(1/(sum_neg_word+len(neg_word_prob)))


    if pos_prob > neg_prob:
        return 1
    else:
        return 0 

In [ ]:
# Test the model on test set and report Accuracy
correct = 0
total = 0
for i, sentence in enumerate(test_sentences):

    if test_labels[i] == modelNB(sentence):
        correct += 1
    total += 1

print(f'Accuracy : {(correct/total)*100}')

# *LSTM* based Classifier



In [12]:
# Hyperparameters of the model
vocab_size = 3000
oov_tok = '<OOK>'
embedding_dim = 100
max_length = 131 # based on avg sentence length
padding_type='post'
trunc_type='post'

In [13]:
# tokenize sentences
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

# convert train dataset to sequence and pad sequences
train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, padding='post', maxlen=max_length)

# convert Test dataset to sequence and pad sequences
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, padding='post', maxlen=max_length)

In [14]:
# model initialization
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# compile model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 131, 100)          300000    
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               84480     
_________________________________________________________________
dense (Dense)                (None, 24)                3096      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 25        
Total params: 387,601
Trainable params: 387,601
Non-trainable params: 0
_________________________________________________________________


In [15]:
num_epochs = 5
history = model.fit(train_padded, train_labels, 
                    epochs=num_epochs, verbose=1, 
                    validation_split=0.1)

Epoch 1/5
1125/1125 [==============================] - 5316s 5s/step - loss: 0.4691 - accuracy: 0.7552 - val_loss: 0.3399 - val_accuracy: 0.8593
Epoch 2/5
1125/1125 [==============================] - 165s 147ms/step - loss: 0.2673 - accuracy: 0.8922 - val_loss: 0.3015 - val_accuracy: 0.8770
Epoch 3/5
1125/1125 [==============================] - 189s 168ms/step - loss: 0.2185 - accuracy: 0.9142 - val_loss: 0.3101 - val_accuracy: 0.8763
Epoch 4/5
1125/1125 [==============================] - 188s 167ms/step - loss: 0.1760 - accuracy: 0.9321 - val_loss: 0.3311 - val_accuracy: 0.8630
Epoch 5/5
1125/1125 [==============================] - 199s 177ms/step - loss: 0.1465 - accuracy: 0.9469 - val_loss: 0.3540 - val_accuracy: 0.8640


In [21]:
# Calculate accuracy on Test data
# Get probabilities
prediction = model.predict(test_padded)

# Get labels based on probability 1 if p>= 0.5 else 0
pred_labels = (prediction>=0.5).astype(int)

# Accuracy : one can use classification_report from sklearn
from sklearn.metrics import classification_report
target_names = ['positive', 'negative']
print(classification_report(test_labels, pred_labels, target_names=target_names))

              precision    recall  f1-score   support

    positive       0.84      0.89      0.87      5000
    negative       0.89      0.83      0.86      5000

    accuracy                           0.86     10000
   macro avg       0.86      0.86      0.86     10000
weighted avg       0.86      0.86      0.86     10000



## Get predictions for random examples

In [24]:
# reviews on which we need to predict
sentence = ["The movie was really really good, higly recommend", 
            "terrible terrible piece of art",
            "caught my heart by surprise",
            "acting was good but story made it look even taht baad"]

# convert to a sequence
sequences = tokenizer.texts_to_sequences(sentence)

# pad the sequence
padded = pad_sequences(sequences, padding='post', maxlen=max_length)

# Get probabilities
pred = model.predict(padded)
print(pred)

# Get labels based on probability 1 if p>= 0.5 else 0
pred_labels = (pred>=0.5).astype(int)

# 0 means negative
print(f'Predicted classes of sentences are : {pred_labels}')

[[0.5233905 ]
 [0.4030612 ]
 [0.9637621 ]
 [0.32913542]]
Predicted classes of sentences are : [[1]
 [0]
 [1]
 [0]]
